In [ ]:
# Databricks notebook source
# MAGIC %md
# MAGIC # Customer 360 Data Processing
# MAGIC This notebook processes customer data from Unity Catalog tables to generate a comprehensive customer profile.

# COMMAND ----------

import logging
from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# COMMAND ----------

try:
    # Step 1: Data Selection and Filtering
    logger.info("Loading and filtering data from Unity Catalog tables.")
    
    demographics_df = spark.table("genai_demo.jnj.demographics").select(
        "Customer_ID", "Customer_Name", "Email", "Phone_Number", "Address", "City",
        "State", "Postal_Code", "Date_of_Birth", "Gender", "Marital_Status",
        "Occupation", "Income_Level", "Customer_Segment"
    )

    claims_df = spark.table("genai_demo.jnj.claims").select(
        "Claim_ID", "Policy_ID", "Claim_Date", "Claim_Type", "Claim_Status",
        "Claim_Amount", "Claim_Payout"
    )

    policy_df = spark.table("genai_demo.jnj.policy").select(
        "policy_id", "customer_id", "policy_type", "policy_status", "policy_start_date",
        "policy_end_date", "policy_term", "policy_premium", "total_premium_paid",
        "renewal_status", "policy_addons"
    )

# COMMAND ----------

    # Step 2: Data Integration
    logger.info("Joining demographics and policy data on Customer_ID.")
    customer_policy_df = demographics_df.join(
        policy_df, demographics_df.Customer_ID == policy_df.customer_id, "inner"
    ).drop(policy_df.customer_id)

    logger.info("Joining the result with claims data on Policy_ID.")
    integrated_df = customer_policy_df.join(
        claims_df, customer_policy_df.policy_id == claims_df.Policy_ID, "inner"
    ).drop(claims_df.Policy_ID)

# COMMAND ----------

    # Step 3: Data Aggregation
    logger.info("Aggregating claims data.")
    aggregated_df = integrated_df.groupBy("Customer_ID").agg(
        F.count("Claim_ID").alias("Total_Claims"),
        F.count("policy_id").alias("Policy_Count"),
        F.max("Claim_Date").alias("Recent_Claim_Date"),
        F.avg("Claim_Amount").alias("Average_Claim_Amount")
    )

# COMMAND ----------

    # Step 4: Custom Calculations
    logger.info("Performing custom calculations.")
    final_df = aggregated_df.withColumn(
        "Age", F.datediff(F.current_date(), F.col("Date_of_Birth")) / 365
    ).withColumn(
        "Claim_To_Premium_Ratio", F.when(F.col("total_premium_paid") != 0, F.col("Claim_Amount") / F.col("total_premium_paid")).otherwise(0)
    ).withColumn(
        "Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)
    ).withColumn(
        "Retention_Rate", F.lit(0.85)
    ).withColumn(
        "Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")
    ).withColumn(
        "Upsell_Potential", F.lit("Premium Vehicle Coverage")
    )

# COMMAND ----------

    # Step 5: Comprehensive Data Consolidation
    logger.info("Loading additional data sources and joining them.")
    scores_df = spark.table("genai_demo.jnj.scores")
    aiml_insights_df = spark.table("genai_demo.jnj.aiml_insights")

    customer_360_df = final_df.join(scores_df, "Customer_ID", "inner").join(
        aiml_insights_df, "Customer_ID", "inner"
    )

# COMMAND ----------

    # Step 6: Output Generation
    logger.info("Writing the consolidated customer profile to CSV.")
    customer_360_df.write.csv("/mnt/dataeconomy/Customer_360.csv", header=True)

except AnalysisException as e:
    logger.error(f"AnalysisException occurred: {e}")
except Exception as e:
    logger.error(f"An unexpected error occurred: {e}")
